# 第02章 — 力

- ニュートンの第二法則: 加速度 = 合力 / 質量。これを積分して速度・位置を更新する。
- 代表的な力: 重力、抗力、摩擦、ばね力（フックの法則）。
- 力は毎フレームベクトルとして蓄積し、積分後にリセットする。
- 力の強さは距離でスケールすることが多い（重力なら距離の二乗に反比例）。

試してみよう: 抗力係数や初期距離を変えて、運動の安定/不安定さを試す。

In [1]:
# ベクトル演算の基本クラス
import math

class Vector:
    def __init__(self, x=0.0, y=0.0):
        self.x, self.y = float(x), float(y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, k):
        return Vector(self.x * k, self.y * k)

    __rmul__ = __mul__

    def mag(self):
        return math.hypot(self.x, self.y)

    def normalize(self):
        m = self.mag()
        if m == 0:
            return Vector()
        return self * (1.0 / m)

    def limit(self, max_mag):
        m = self.mag()
        if m > max_mag:
            return self.normalize() * max_mag
        return self

    def __repr__(self):
        return f"Vector({self.x:.2f}, {self.y:.2f})"


In [2]:
# 力を積分するBodyクラス
class Body:
    def __init__(self, pos, mass=1.0):
        self.pos = pos
        self.vel = Vector(0, 0)
        self.acc = Vector(0, 0)
        self.mass = mass

    def apply_force(self, force):
        # F = m * a なので a = F / m
        self.acc += force * (1 / self.mass)

    def update(self, dt=1.0):
        self.vel += self.acc * dt
        self.pos += self.vel * dt
        self.acc = Vector(0, 0)


In [3]:
# 重力と抗力を合成して移動を計算
def gravity(attractor, mover, G=1.0):
    dir_vec = attractor.pos - mover.pos
    dist = max(0.5, dir_vec.mag())
    strength = G * attractor.mass * mover.mass / (dist * dist)
    return dir_vec.normalize() * strength

attractor = Body(Vector(0, 0), mass=20)
mover = Body(Vector(5, 0), mass=2)

for step in range(5):
    g = gravity(attractor, mover, G=0.8)
    drag = mover.vel * -0.05
    mover.apply_force(g)
    mover.apply_force(drag)
    mover.update(0.5)
    print(f"step {step}: pos={mover.pos} vel={mover.vel}")


step 0: pos=Vector(4.84, 0.00) vel=Vector(-0.32, 0.00)
step 1: pos=Vector(4.51, 0.00) vel=Vector(-0.66, 0.00)
step 2: pos=Vector(3.99, 0.00) vel=Vector(-1.04, 0.00)
step 3: pos=Vector(3.22, 0.00) vel=Vector(-1.53, 0.00)
step 4: pos=Vector(2.08, 0.00) vel=Vector(-2.28, 0.00)


### 追加例: 抗力付きの落下（終端速度の例）

In [4]:
# 抗力付きの1次元落下で終端速度を観察
import math

class Body:
    def __init__(self, mass=1.0, drag_coef=0.2):
        self.v = 0.0
        self.y = 50.0
        self.mass = mass
        self.drag = drag_coef

    def step(self, dt=0.1, g=-9.8):
        # 抗力: Fd = -c * v * |v|
        drag = -self.drag * self.v * abs(self.v)
        acc = (g * self.mass + drag) / self.mass
        self.v += acc * dt
        self.y += self.v * dt

body = Body(mass=1.5, drag_coef=0.3)
for i in range(10):
    body.step()
    print(f"t={i:02d} v={body.v:.3f} y={body.y:.2f}")


t=00 v=-0.980 y=49.90
t=01 v=-1.941 y=49.71
t=02 v=-2.845 y=49.42
t=03 v=-3.664 y=49.06
t=04 v=-4.375 y=48.62
t=05 v=-4.972 y=48.12
t=06 v=-5.458 y=47.58
t=07 v=-5.842 y=46.99
t=08 v=-6.139 y=46.38
t=09 v=-6.366 y=45.74
